0 Modules
=====

In [55]:
import sys
from pathlib import Path

from get_clunis import *
from get_donauts_sat import *
from get_donauts_dof import *

from pandas import merge
from pandas import set_option

from urllib3.exceptions import ReadTimeoutError

sys.path.append("../../src/directorio")

In [ ]:
set_option('display.max_colwidth', None)

1 Directory Structure
=====

In [56]:
DIRS = [
    "./logs",
    "./params",
     "./resources"
]

In [57]:
for d in DIRS: 
    Path(d).mkdir(parents=True, exist_ok=True)

2 Collect Data
=====

In [ ]:
df_clunis = get_clunis()

In [ ]:

try:
    df_donauts = get_donauts_sat()
except ReadTimeoutError:
    df_donauts = get_donauts_dof()


3 Merge Directories
=====

In [120]:

df_oscs = merge(df_clunis, df_donauts, on="rfc", how="outer")

In [121]:
df_oscs = df_oscs.drop_duplicates(["rfc", "cluni"])

In [122]:
# FIXME: Inlcude OSCs with duplicated records.
rfc_duplicados = df_oscs.rfc.value_counts()[df_oscs.rfc.value_counts() > 1].index.tolist()
df_oscs = df_oscs[~df_oscs.rfc.isin(rfc_duplicados)]

In [124]:
only_cluni = ~df_oscs.cluni.isnull() & df_oscs.sat_razon_social.isnull()
only_donaut = df_oscs.cluni.isnull() & ~df_oscs.sat_razon_social.isnull()
cluni_donaut = ~df_oscs.cluni.isnull() & ~df_oscs.sat_razon_social.isnull()
rfc_missing_data = ~( only_cluni | only_donaut | cluni_donaut )

In [126]:
len(df_oscs) == ( 
    len(df_oscs[only_cluni]) 
    + len(df_oscs[only_donaut])
    + len(df_oscs[cluni_donaut])
    + sum(rfc_missing_data)
)

True

In [127]:
df_oscs = df_oscs[~rfc_missing_data].copy()
df_oscs = df_oscs.reset_index(drop=True)

In [128]:
df_oscs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45960 entries, 0 to 45959
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   rfc                   45960 non-null  object        
 1   cluni                 42710 non-null  object        
 2   cluni_razon_social    42710 non-null  object        
 3   cluni_status          42710 non-null  object        
 4   cluni_representation  42710 non-null  object        
 5   osc_legal_form        42710 non-null  object        
 6   created_at            42708 non-null  datetime64[ns]
 7   sat_razon_social      9605 non-null   object        
 8   dof_year              9697 non-null   object        
dtypes: datetime64[ns](1), object(8)
memory usage: 3.2+ MB


In [136]:
df_oscs[df_oscs.cluni_status == 'activa'].sample(5)

,rfc,cluni,cluni_razon_social,cluni_status,cluni_representation,osc_legal_form,created_at,sat_razon_social,dof_year
25315,VSC1312066X8,VSC1312062501X,Voluntas al Servicio de la Comunidad,activa,vigente,asociación civil,2013-12-06,NaN,NaN
29042,COS081217H90,COS08121720T4A,Comité Oaxaqueño de Sanidad e Inocuidad Acuícola,activa,vencida,asociación civil,2008-12-18,NaN,NaN
12931,FER460415SF8,FER4604152601C,Fundación Esposos Rodriguez,activa,vigente,sociedad civil,1946-08-02,"Fundación Esposos Rodríguez, A.B.P.",2020
3266,LIE070312UV5,LIE07031209010,Laboratorio de Investigaciones Escénico Musicales,activa,vigente,asociación civil,2007-03-12,NaN,NaN
20174,IZA120914TU7,IZA1209143201L,Inclusión Zacatecas,activa,vigente,asociación civil,2012-09-14,"Inclusión Zacatecas, A.C.",2020
